# Exercises

Sourcing the Zillow database as the data source, create a jupyter notebook named explore_zillow and do the following:

Ask at least 5 questions about the data, keeping in mind that your target variable is logerror. e.g. Is logerror significantly different for properties in LA County vs Orange County vs Ventura County?

Answer those questions through a mix of statistical tests and visualizations.

In [1]:
# standard imports
# ignore warnings
import os
from env import get_connection
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

# possible modules, may just wrangle like before
import wrangle
import summarize

In [2]:
df = wrangle.wrangle_zillow()

In [3]:
df.shape

(35446, 41)

# 1. Is there a relationship between logerror and the property's location (e.g., county or city)? You can use visualizations like box plots or bar charts to compare logerror across different locations.

In [ ]:
df. 

# 2. How does logerror vary with property size (e.g., square footage)? You can create scatter plots to visualize this relationship.

# 3. Is there a correlation between logerror and property age? You can calculate the correlation coefficient and plot a scatter plot to explore this.

# 4. Does logerror vary with the time of year (e.g., seasonality)? You can group the data by month and visualize logerror trends over time.

# 5. Is there a difference in logerror for different property types (e.g., single-family homes vs. condos)? You can use groupings and statistical tests to compare logerror between property types.

# Bonus:

Compute the mean(logerror) by zipcode and the overall mean(logerror). Write a loop that will run a t-test between the overall mean and the mean for each zip code. We want to identify the zip codes where the error is significantly higher or lower than the expected error.